# Install conda on your Colab environment

Ignore this first cell if you are running the notebook in a local environment.

One can still run it locally but it will have no effect.

In [1]:
# Run this cell first - it will install a conda distribution (mamba)
# on your Drive then restart the kernel automatically 
# (don't worry about the crashing/restarting kernel messages)
# It HAS to be runned FIRST everytime you use the notebook in colab

import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

# Set up your Colab or local environment
# Then import libraries

Run this cell in both cases of use (local or Colab)

In [1]:
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    
    # Check everything is fine with conda in Colab
    import condacolab
    condacolab.check()
    
    # Mount your drive environment in the colab runtime
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    
    # Change this variable to your path on Google Drive to which the repo has been cloned
    # If you followed the colab notebook 'repo_cloning.ipynb', nothing to change here
    repo_path_in_drive = '/content/drive/My Drive/Github/amn_release/'
    # Change directory to your repo cloned in your drive
    DIRECTORY = repo_path_in_drive
    os.chdir(repo_path_in_drive)
    # Copy the environment given in the environment_amn_light.yml
    !mamba env update -n base -f environment_amn_light.yml
    
    # This is one of the few Colab-compatible font
    font = 'Liberation Sans'
    
else:
    
    # In this case the local root of the repo is our working directory
    DIRECTORY = './'
    font = 'arial'

# printing the working directory files. One can check you see the same folders and files as in the git webpage.
print(os.listdir(DIRECTORY))

from Library.Build_Model_KO import *

# We declare this function here and not in the
# function-storing python file to modify it easily
# as it can change the printouts of the methods
def printout(filename, Stats, model, time): 
    # printing Stats
    print('Stats for %s CPU-time %.4f' % (filename, time))
    print('R2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    print('Q2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.test_objective[0], Stats.test_objective[1],
           Stats.test_loss[0], Stats.test_loss[1]))

['README.md', 'Duplicate_Model.ipynb', 'Dataset_experimental', 'Tutorial.ipynb', '.ipynb_checkpoints', '.git', 'Build_Model_RC.ipynb', 'Build_Model_AMN_KO.ipynb', 'biolog_simulations.npy', 'environment_amn_light.yml', 'Build_Experimental.ipynb', 'Reservoir', 'Build_Model_MM.ipynb', 'Dataset_model', 'Figures.ipynb', 'Result', 'Figures', '.gitignore', 'LICENSE', 'Build_Model_ANN_Dense.ipynb', 'Build_Dataset.ipynb', 'Library', 'Dataset_input', 'Functions', 'Build_Dataset_KO.ipynb', 'environment_amn.yml', 'Build_Model_AMN.ipynb', '.DS_Store']


In [2]:
Maxloop, Q2, PRED = 1, [], []

for Nloop in range(Maxloop):
    # What you can change
    seed = Nloop+1
    np.random.seed(seed=seed)
    trainname = 'biolog_iML1515_EXP_UB'
    timestep = 0
    # End of What you can change
  
    # Create model 100% for training 0% for testing
    trainingfile = DIRECTORY+'Dataset_model/'+trainname
    model = Neural_Model(trainingfile = trainingfile, 
              objective=['BIOMASS_Ec_iML1515_core_75p37M'],
              model_type = 'AMN_QP',
              scaler = True,
              timestep = timestep, learn_rate=0.001,
              n_hidden = 1, hidden_dim = 500,
              # n_hidden = 1, hidden_dim = 500,
              #train_rate = 1.0e-2,
              # epochs = 1000, xfold = 10,
              batch_size=100,
              epochs=200, xfold=10,
              verbose=True)

    # Train and evaluate
    reservoirname = trainname +'_'+model.model_type
    reservoirfile = DIRECTORY+'Reservoir/'+reservoirname
    start_time = time.time()
    reservoir, pred, stats, _ = train_evaluate_model(model, verbose=2)
    delta_time = time.time() - start_time

    # Printing cross-validation results
    printout(reservoirname, stats, model, delta_time)
    r2 = r2_score(model.Y, pred[:,0], multioutput='variance_weighted')
    print('Iter', Nloop, 'Collated Q2', r2)
    Q2.append(r2)
    PRED.append(pred[:,0])
    
# Save in Result folder
Q2, PRED = np.asarray(Q2), np.asarray(PRED)
print('Averaged Q2 = %.4f (+/- %.4f)' % (np.mean(Q2), np.std(Q2)))
filename = DIRECTORY+'Result/'+reservoirname+'_Q2.csv'
np.savetxt(filename, Q2, delimiter=',')
filename = DIRECTORY+'Result/'+reservoirname+'_PRED.csv'
np.savetxt(filename, PRED, delimiter=',')

number of reactions:  3682 1
number of metabolites:  1877
filtered measurements size:  1
AMN scaler 50.0
QP input shape (17400, 430) (17400, 4)
-------train [    0     1     2 ... 17397 17398 17399]
-------test  [    4     5     6 ... 17384 17385 17391]
Dense layer n_hidden, hidden_dim, output_dim, activation, trainable: 1 500 3682 relu True
AMN output shapes for PoutV, SV, PinV, Vpos, V, outputs (None, 1) (None, 1) (None, 1) (None, 1) (None, 3682) (None, 7368)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 430)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 500)          215500      input_1[0][0]                    
________________________